# RAG 과제

## 1. 문서 로드

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/CN7N_2026_ko_KR.pdf"

print("문서 로드중")

loader = PyPDFLoader(file_path)
docs = loader.load()
docs[:3]

문서 로드중


[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2025-06-17T12:11:08+09:00', 'moddate': '2025-06-17T12:11:08+09:00', 'source': '../data/CN7N_2026_ko_KR.pdf', 'total_pages': 436, 'page': 0, 'page_label': '1'}, page_content='주 의\n차량이 고장나거나 손상될 우려가 있는 경우의 주의 표\n시입니다.\n 경 고\n사람이 다치거나 사망의 우려가 있는 경우의 경고 표시\n입니다.\ni  알아두기\n차량 용어 또는 추가 설명이 필요한 정보 표시입니다.\n안전을 위해 반드시 지켜야 하는 금지 표시입니다.\n본 취급설명서는 고객 및 차량의 안전과 관련한 심각한 위험과 제품 사용에 대한 올바른 정보를 사\n전에 알리는 안전경고 표시입니다. 지시사항은 반드시 숙지하여 지켜주십시오.\n경고, 주의표시\n안전 및 차량 손상 경고\n경고, 주의가 있는 문장 및 진하게 표시되어 있는 부분은 \n특히 유념하십시오.\n선택 또는 미장착 사양표시 \n< >\n본 취급설명서에는 모든 트림모델 및 선택사양을 포함\n하여 설명하고 있습니다.\n따라서 고객님 차량에 장착되지 않은 사양이 설명될 수 \n있습니다.'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2025-06-17T12:11:08+09:00', 'moddate': '2025-06-17T12:11:08+09:00', 'source': '../data/CN7N_2026_ko_KR.pdf', 'total_pages': 436, 'page': 1, 'page_label': '2'}, page_content='내용 찾기 방법 설명\n에어컨\n사용방법이\n어디지...\n이 스위치가\n뭐지?\n점검방법이\n어느 장에 있지\n목

In [2]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

In [3]:
def tiktoken_len(text):
    return len(encoding.encode(text))

## 2. splitter

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,               # 토큰 단위
    chunk_overlap=100,
    length_function=tiktoken_len  # 토큰 수로 길이 측정
)
chunks = splitter.split_documents(docs)
print(f"총 chunk 수: {len(chunks)}")

총 chunk 수: 588


## 3. vectorDB 저장 및 불러오기

In [5]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
persist_directory = "../vectorStore/hyundai_car"
collection_name = "hyundai_car"

In [ ]:
# # 빈 벡터스토어 생성
# vectorstore = Chroma(
#     collection_name=collection_name,
#     persist_directory=persist_directory,
#     embedding_function=embedding
# )

# # 안전하게 나눠서 임베딩 생성 및 저장
# batch_size = 100  # 또는 50~200 사이 적당히
# for i in range(0, len(chunks), batch_size):
#     batch = chunks[i:i+batch_size]
#     vectorstore.add_documents(batch)

In [9]:
load_vectorStore = Chroma(
    persist_directory = persist_directory,
    collection_name = collection_name,
    embedding_function = embedding
)

In [10]:
load_vectorStore

## 4. Retriever

In [11]:
retriever = load_vectorStore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 30}
)

## 5. Reranker

In [12]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512
    }
)

reranker = CrossEncoderReranker(
    model = hf_ce,
    top_n = 10
)

## 6. Retriever -> Reranker

In [13]:
from langchain.retrievers import ContextualCompressionRetriever

comp_retriever = ContextualCompressionRetriever(
    base_retriever = retriever,
    base_compressor = reranker
)

## 7. Reorder

In [14]:
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()

In [15]:
def format_docs(docs):
    result = []
    for item in docs:
        result.append(item.page_content)
    return "\n\n---\n\n".join(result)

## 8. chain 만들기

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """
        주어진 컨텍스트에 근거해 간결하고 정확하게 답하라.
        특정 문제 사항에 대한 해결책을 단계별로 안내하도록 해라.
        정보 누락이 없도록 특정 주제 요약 및 정리해라.
        답변 출처를 꼭 표시 하도록 해라. 페이지도 같이 표시해라.
     
        [컨텍스트]
        {context}
    """),
    ("human", "{question}")
])

# 2. 모델 설정
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 3. outputparser
outputparser = StrOutputParser()

# 4. 체인 설정
chain = rag_prompt | model | outputparser
chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        주어진 컨텍스트에 근거해 간결하고 정확하게 답하라.\n        특정 문제 사항에 대한 해결책을 단계별로 안내하도록 해라.\n        정보 누락이 없도록 특정 주제 요약 및 정리해라.\n        답변 출처를 꼭 표시 하도록 해라. 페이지도 같이 표시해라.\n\n        [컨텍스트]\n        {context}\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B12744AE50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B18A789790>, root_client=<openai.OpenAI object at 0x000001B18A7656D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001B18A788110>, model_name='gpt-4.1-mini'

In [26]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

rag_chain = (
    {
        "docs" : RunnableLambda(lambda x: comp_retriever.invoke(x["question"])),
        "question" : RunnablePassthrough()
    }
    | RunnableLambda(lambda x : {
        "context" : format_docs(reorder.transform_documents(x["docs"])),
        "question" : x["question"]
    })
    | chain
)
rag_chain

{
  docs: RunnableLambda(...),
  question: RunnablePassthrough()
}
| RunnableLambda(...)
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        주어진 컨텍스트에 근거해 간결하고 정확하게 답하라.\n        특정 문제 사항에 대한 해결책을 단계별로 안내하도록 해라.\n        정보 누락이 없도록 특정 주제 요약 및 정리해라.\n        답변 출처를 꼭 표시 하도록 해라. 페이지도 같이 표시해라.\n\n        [컨텍스트]\n        {context}\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B12744AE50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B18A789790>, root_client=<openai.OpenAI object at 0x000001B18A7656D0>, roo

In [27]:
question = "차의 결함은 어떻게 확인하나요?"

In [28]:
result = rag_chain.invoke({
    "question" : question
})

In [31]:
print(f"질문 : {question}")
print(f"답변 : {result}")

질문 : 차의 결함은 어떻게 확인하나요?
답변 : 차의 결함 확인 방법은 다음과 같습니다:

1. **경고등 및 경고문 확인**  
   - 차량 계기판에 경고등이 켜지거나 경고문이 표시되면 즉시 확인하십시오.  
   - 경고등이 켜지면 당사 직영 하이테크센터나 블루핸즈에서 점검을 받는 것이 안전합니다.  
   (출처: 안전 및 차량 손상 경고, 페이지 7-28, 7-37)

2. **운행 중 이상 증상 관찰**  
   - 가속 페달 조작 시 차량이 갑자기 빠르게 출발하거나, 브레이크 작동이 원활하지 않을 경우 주의하십시오.  
   - 자동 정차 기능(Auto Hold) 작동 이상 시 해제 후 주행하십시오.  
   (출처: 안전 및 차량 손상 경고, 페이지 7-28)

3. **주기적 정기 점검**  
   - 차량 하체, 휠, 브레이크액 및 패드 상태를 정기적으로 점검하여 이상 유무를 확인하십시오.  
   - 특히 겨울철 염화칼슘 도포 지역 주행 후에는 부식 여부를 점검해야 합니다.  
   (출처: 정기 점검, 페이지 9-42)

4. **센서 및 보조장치 이상 확인**  
   - 후측방 충돌방지 보조, 후방 주차 충돌방지 보조 등 운전자 보조 시스템이 정상 작동하지 않거나 경고음이 울릴 경우 점검이 필요합니다.  
   - 강한 전자파나 충격으로 센서가 오작동할 수 있으니 센서 상태를 확인하십시오.  
   (출처: 운전자 보조, 페이지 7-37, 7-99)

5. **차량 외관 및 부품 손상 점검**  
   - 차체 표면에 깊은 흠집이나 금속 노출 부위가 있으면 빠르게 녹이 슬 수 있으므로 신속히 수리하십시오.  
   - 무광 컬러 차량은 부분 수리가 어려우니 전문 정비소 방문을 권장합니다.  
   (출처: 표면 손상의 정비, 페이지 7-99)

요약: 차량 결함은 경고등 및 경고문 확인, 운행 중 이상 증상 관찰, 정기 점검, 센서 및 보조장치 이상 확인, 외관 손상 점검을 통해 확인하며, 이상 발견 시 반드시 당사 직영 하이테크센터